In [ ]:
import numpy as np
import random as rd
import time
import matplotlib.pyplot as plt

In [ ]:
'''
Receives a String filename with name of the csv file of the dataset.

Returns: a numpy array with the loaded data
'''
def serialReadFile(filename):

    # Read the CSV file into a numpy array (we skip the first row that is the labels of columns)
    data = np.loadtxt(filename, delimiter=',', skiprows=1)
    
    # We exclude the first colomn that is the predicted output label
    return data[:, 1:]

In [ ]:
'''
Receives a list of d-dimensional tuples called centroids, representing the current state
of the centroids, and a d-dimensional tuple x which represents the datum to be assigned to a cluster.

Returns: an integer with the index in centroids of the closest centroid to x
'''
def serialAssign2cluster(x, centroids):
    
    # Initialisation of the distances from x to the centroids
    distances = [0 for i in range(len(centroids))]
    
    # For each centroid, we compute the euclidian distance to x
    for c in range(len(centroids)):
        for i in range(784):
            distances[c] += (x[i] - centroids[c][i]) ** 2
        distances[c] = np.sqrt(distances[c])
        #distances[c] **= 0.5
    
    # We look for the smallest distance which corresponds to the nearest centroid
    dmin = distances[0]
    for d in distances[1:]:
        if d < dmin:
            dmin = d
            
    return distances.index(dmin)

In [ ]:
'''
Performs the serialized K-Means algorithm on the dataset X, grouping the instances into K different
clusters. The number of iterations of the method to be executed is n_iter. The initialization of the centroids will be
random, sampled from a standard normal distribution. It returns a list of length K with the d-dimensional centroids
computed
'''
# Following the implementation described in the slides        
def serialKMeans(X, K, n_iter):
    # Generate K random tuples with 28000 elements each (len of each x)
    centroids = [tuple(np.random.randn(784)) for i in range(K)]
    
    for it in range(n_iter):
        
        print("it : ", it)
        
        # Initialize the "clusters" of clusters
        clusters = [[] for i in range(K)]
        
        # Assign each sample to a cluster
        for x in X:
            x = tuple(x) # According to specs, serialAssign2cluster needs x as a d-dimensional tuple
            closest_c = serialAssign2cluster(x, centroids)
            clusters[closest_c].append(x)
        
        # Compute the new centroids
        # Initialise to 0
        centroids = [list(0 for i in range(784)) for i in range(K)]
        
        # Add each value of each sample in the same cluster
        for j in range(K):
            for x in clusters[j]:
                for k in range(784):
                    centroids[j][k] += x[k]
                    
        # Divide each value of each centroid by the number of sample in the cluster
        for j in range(K):
            for k in range(784):
                centroids[j][k] /= len(clusters[j])
        
        # Convert centroids from lists (mutable) to tuples (immutable)
        for i in range(K):
            centroids[i] = tuple(centroids[i])
        
        # Plot
        i=1
        plt.figure(figsize = (20,5))
        for c in centroids:
            ax = plt.subplot(1, K, i)
            c = np.asarray(c)
            plt.imshow(c.reshape(28,28),cmap='gray')
            ax.set_axis_off() 
            i+=1
        plt.show()
                
    return centroids

In [ ]:
'''
Performs the serialized K-Means algorithm on the dataset X, grouping the instances into K different
clusters. The number of iterations of the method to be executed is n_iter. The initialization of the centroids will be
random, sampled from a standard normal distribution. It returns a list of length K with the d-dimensional centroids
computed
'''
# Modifications made by ourselves, difference is that we put the indexes
# rather than samples in clusters so it's faster

def MyserialKMeans(X, K, n_iter):
    # Generate K random tuples with 28000 elements each (len of each x)
    centroids = [tuple(np.random.randn(784)) for i in range(K)]
    
    for it in range(n_iter):
        print("it : ", it)
        
        # Initialize the clusters
        clusters = [[] for i in range(K)]
        
        # Initialize sample number to add to clusters
        sample_nbr = 0
        
        # Assign each sample to a cluster
        for x in X:
            x = tuple(x) # According to specs, serialAssign2cluster needs x as a d-dimensional tuple
            closest_c = serialAssign2cluster(x, centroids)
            x = list(x)
            clusters[closest_c].append(sample_nbr)
            sample_nbr += 1
        
        # Compute the new centroids
        
        # Initialise to 0
        centroids = [list(0 for i in range(784)) for i in range(K)]
        
        # Add each value of each sample in the same cluster
        for i in range(K):
            for j in range(len(clusters[i])):
                for k in range(784):
                    centroids[i][k] += X[clusters[i][j]][k]
        
        # Divide each value of each centroid by the number of sample in the cluster
        for i in range(K):
            for k in range(784):
                centroids[i][k] /= len(clusters[i])
                
        # Convert centroids from lists (mutable) to tuples (immutable)
        for i in range(K):
            centroids[i] = tuple(centroids[i])
                
    return centroids

In [ ]:
filename = "tot_mnist_shuf.csv"
K = 10
start_time = time.time()

data = serialReadFile(filename)
centroids = serialKMeans(data, K, 10)

end_time = time.time()
print("Execution time: ", end_time - start_time)

In [ ]:
sample = 2
image = data[sample]
fig = plt.figure
plt.imshow(image.reshape(28,28),cmap='gray')